In [2]:
from utils import DataPoint, PredictionModel, ScorerStepByStep
import pandas as pd
import numpy as np

/home/evenediktov/projects/wnn/.env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
all_data = pd.read_parquet('datasets/train.parquet')

In [4]:
all_data.head()

,seq_ix,step_in_seq,need_prediction,0,1,2,3,4,5,6,...,22,23,24,25,26,27,28,29,30,31
0,0,0,0,-0.080082,1.324564,0.637730,-1.091486,-0.859104,1.291364,-1.046394,...,0.876881,-1.425938,-0.104571,1.253508,-1.155514,-0.907224,1.361777,0.602751,1.265182,-0.495940
1,0,1,0,-1.243274,0.398425,-1.810485,-0.427042,0.303473,0.304587,-0.392796,...,0.669286,-0.720985,-1.144171,0.724269,-1.259564,0.181201,0.529757,-0.076843,1.111895,-1.171435
2,0,2,0,-0.124680,1.225580,0.139482,-0.967710,-0.769101,1.208697,-1.218701,...,0.742988,-1.381406,0.043121,1.205969,-1.062414,-0.739191,1.266046,0.856712,1.205016,-0.417057
3,0,3,0,-1.264455,0.494485,-1.041663,-0.587907,0.203893,0.437923,-0.479116,...,0.808542,-0.777047,-1.163911,0.801497,-1.280313,-0.090955,0.636906,-0.213926,1.121746,-1.364489
4,0,4,0,-0.882663,-0.223168,-1.229349,0.029471,0.459377,-0.232340,0.113789,...,0.286553,-0.114436,-0.730509,0.301325,-0.782454,0.486104,-0.172012,-0.016936,0.674722,-0.740237


In [8]:
total_num = len(all_data['seq_ix'].unique())
print('Number of sequences:', total_num)
print('Number of steps in each sequence:', all_data.groupby('seq_ix')['step_in_seq'].count())




Number of sequences: 517
Number of steps in each sequence: seq_ix
0      1000
1      1000
2      1000
3      1000
4      1000
       ... 
512    1000
513    1000
514    1000
515    1000
516    1000
Name: step_in_seq, Length: 517, dtype: int64


In [10]:
all_data.iloc[0]

seq_ix             0.000000
step_in_seq        0.000000
need_prediction    0.000000
0                 -0.080082
1                  1.324564
2                  0.637730
3                 -1.091486
4                 -0.859104
5                  1.291364
6                 -1.046394
7                 -0.378080
8                  0.282473
9                  1.229306
10                 0.963248
11                 0.943299
12                 1.222109
13                -1.014774
14                 0.771474
15                -1.107038
16                -1.125827
17                -0.708075
18                 1.163483
19                 0.053451
20                -0.272005
21                 0.153156
22                 0.876881
23                -1.425938
24                -0.104571
25                 1.253508
26                -1.155514
27                -0.907224
28                 1.361777
29                 0.602751
30                 1.265182
31                -0.495940
Name: 0, dtype: floa

In [11]:
# Split sequences into train and test
np.random.seed(42)  # For reproducibility

# Get unique sequence indices
unique_seq_ixs = all_data['seq_ix'].unique()
num_sequences = len(unique_seq_ixs)

# Split sequences into train (80%) and test (20%)
test_size = 0.2
num_test = int(num_sequences * test_size)
test_seq_ixs = np.random.choice(unique_seq_ixs, size=num_test, replace=False)
train_seq_ixs = np.setdiff1d(unique_seq_ixs, test_seq_ixs)

# Split the data
train_data = all_data[all_data['seq_ix'].isin(train_seq_ixs)].copy()
test_data = all_data[all_data['seq_ix'].isin(test_seq_ixs)].copy()

print(f"Total sequences: {num_sequences}")
print(f"Train sequences: {len(train_seq_ixs)} ({len(train_seq_ixs)/num_sequences*100:.1f}%)")
print(f"Test sequences: {len(test_seq_ixs)} ({len(test_seq_ixs)/num_sequences*100:.1f}%)")
print(f"\nTrain data rows: {len(train_data)}")
print(f"Test data rows: {len(test_data)}")


Total sequences: 517
Train sequences: 414 (80.1%)
Test sequences: 103 (19.9%)

Train data rows: 414000
Test data rows: 103000


In [ ]:
# Save train and validation splits
train_data.to_parquet('datasets/train_split.parquet', index=False)
test_data.to_parquet('datasets/val.parquet', index=False)

Saved train_split.parquet and val.parquet
